# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

### Import necessary libraries

In [10]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
import numpy.linalg as LA

In [11]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [12]:
config = pd.read_csv('config.csv').iloc[0]

In [13]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [14]:
user = config['id']
k_value = config['k']

### Read the Data

In [15]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

In [16]:
# your code here

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [17]:
user_likes = dict()
dis_image=set()

In [18]:
for line in appreciate_data:
    line = line.strip()
    
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])

    if user_id not in user_likes:
        user_likes[user_id] = list()
    
    user_likes[user_id].append(item_id)
    dis_image.add(item_id)

In [19]:
len(user_likes)

63497

In [20]:
sp_matrix=np.zeros((len(user_likes),len(dis_image)),dtype=np.int8)

In [21]:
sp_matrix.shape

(63497, 178788)

In [22]:
user_index=dict()
image_index=dict()
count=0
for u in user_likes.keys():
  user_index[u]=count
  count+=1
count=0
for image in dis_image:
  image_index[image]=count
  count+=1


In [23]:
row=list()
colunm=list()
data=list()

In [24]:
for u,images in user_likes.items():
  for image in images:
    data.append(1)
    row.append(user_index[u])
    colunm.append(image_index[image])

In [25]:
sp_matrix=csr_matrix((data,(row,colunm)))

In [26]:
sp_matrix.shape

(63497, 178788)

In [27]:
svd = TruncatedSVD(n_components=1000, random_state=22)


In [28]:
sp_matrix=svd.fit_transform(sp_matrix)

In [29]:
sp_matrix

array([[ 1.96726824e+00,  9.11994141e-01,  5.44346433e-01, ...,
        -4.46532705e-01, -3.28913900e-01, -2.77623744e-01],
       [ 1.37385233e+01, -2.98822497e+00,  5.37449163e+00, ...,
         1.61696718e-01,  9.99176702e-04,  3.03056155e-01],
       [ 4.80799880e-01,  1.93681780e-01,  1.70789050e-01, ...,
        -1.58394434e-01, -2.90307222e-01,  1.01585216e-01],
       ...,
       [ 3.27848620e-02,  3.21566830e-02, -2.08325872e-02, ...,
        -8.52303324e-03, -9.88617718e-03, -1.83948522e-02],
       [ 3.43684984e-04, -6.94795553e-04,  9.35302334e-04, ...,
         2.99442388e-02, -3.48051656e-03, -2.16404347e-03],
       [ 5.85482350e-02, -3.28784637e-02, -6.13980403e-02, ...,
        -1.27816469e-02,  8.51654384e-03, -5.02181572e-03]])

In [30]:
cosine = lambda a,b: np.dot(a,b)/(LA.norm(a) * LA.norm(b))

In [31]:
index_user=dict()
for u,i in user_index.items():
  index_user[i]=u

In [32]:
def is_subset(recom,user_list):
  return len(recom) == len(set(recom) & set(user_list))

### Use KNN after Collaborative Filtering to find nearest neighbors

In [33]:
def neighbors(user,k_value):
  ans=list()
  for i in range(len(sp_matrix)):
    if i == user_index[user]:
      continue
    sim = cosine(sp_matrix[i],sp_matrix[user_index[user]])
    ans.append((sim,index_user[i]))
  ans.sort(reverse=True)
  temp=[x[1] for x in ans]

  # print(ans[:k_value])
  return temp

### Open the output file to write all the lines to the file

In [34]:
outFile = open(config['output_file'], 'w')
ans = neighbors(user,k_value)
count=0
for n_user in ans:
    # user_id = list(user_likes.keys())[n_user]
    user_id=n_user
    if count == k_value:
      break
    if is_subset(user_likes[n_user],user_likes[user]):
      continue
    print(n_user)
    count+=1
    for item_id in user_likes[user_id]:
      if item_id not in user_likes[user]:
        outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()

1480429
2452817
2539765
148706
1525134


In [35]:
# print(neighbors(user,k_value))

In [37]:
# def neighbors1(user,k_value):
#     """ returns an iterable object (like list or generator) """
#     ans=[]
#     for users,key in user_likes.items():
#       if user != users:
#         inter=len(list(set(key) & set(user_likes[user])))
#         # temp=set(key) & set(user_likes[user])
#         # inter=len(list(temp))
#         union= len(key) + len(user_likes[user]) - inter
#         # user_likes[users]=list(set(user_likes[user]) - temp)
#         ans.append((inter/union,users))

#     ans.sort(reverse=True)
#     print(ans[:k_value])
#     return [x[1] for x in ans[:k_value]]
  
# print(neighbors1(user,k_value))

[(0.07142857142857142, 2452817), (0.057692307692307696, 1480429), (0.052083333333333336, 3230001), (0.050505050505050504, 3138371), (0.048672566371681415, 843447)]
[2452817, 1480429, 3230001, 3138371, 843447]
